In [1]:
import pandas as pd
import numpy as np
import time
from scipy.spatial.distance import minkowski

In [2]:
conjunto = pd.read_csv('column_3C.dat', header=None, sep='\s+')
conjunto.head()

,0,1,2,3,4,5,6
0,63.03,22.55,39.61,40.48,98.67,-0.25,DH
1,39.06,10.06,25.02,29.00,114.41,4.56,DH
2,68.83,22.22,50.09,46.61,105.99,-3.53,DH
3,69.30,24.65,44.31,44.64,101.87,11.21,DH
4,49.71,9.65,28.32,40.06,108.17,7.92,DH


In [3]:
sep = {'20/80': {'train': 0.2, 'test': 0.8},
       '30/70': {'train': 0.3, 'test': 0.7},
       '50/50': {'train': 0.5, 'test': 0.5},
       '70/30': {'train': 0.7, 'test': 0.3},
       '80/20': {'train': 0.8, 'test': 0.2}}

### Classificador Vizinho Mais Próximo (distância de Minkowski de ordens m ∈ {0,5; 2/3; 1; 3/2; 2; 5/2})

In [4]:
resultados = {}
for s in sep.keys():
    resultados[s] = {}

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    for m in [0.5, 2/3, 1, 3/2, 2, 5/2]: # diferentes ordens da distância de Minkowski

        print(f'\n\nSeparação {s} e distância de Minkowski de ordem {m}\n')

        Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
        inicio = time.perf_counter()

        for rep in range(100): # 100 rodadas de treino
            #print(f'Rodada de número {rep + 1}')

            conjunto = conjunto.sample(frac=1).reset_index(drop=True) # embaralha o conjunto
            x, y = conjunto.iloc[:, :-1], conjunto.iloc[:, -1]

            # conjunto de treino
            x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
            
            # conjunto de teste
            x_tst, y_tst = x.iloc[N_trn+1:, :], y.iloc[N_trn+1:]

            # loop que itera sobre o conjunto de teste
            acerto = 0 # contador de acertos
            for i in x_tst.index:
                x_new = np.array(x_tst.loc[i, :]) # nova amostra a ser classificada
                y_new = y_tst.loc[i] # classe da nova amostra

                dist_array = []
                for j in x_trn.index:
                    dist_array.append(minkowski(x_new, np.array(x_trn.loc[j, :]), p=m)) # distância de Minkowski de ordem m
                
                j_min = np.argmin(dist_array) # índice do menor valor de distância
                if y_trn.loc[j_min] == y_new: # verifica se a classificação foi correta
                    acerto += 1
        
            Pacerto.append(acerto / len(x_tst)) # calcula a porcentagem de acertos
        fim = time.perf_counter()

        resultados[s][m] = {'media': np.mean(Pacerto),
                           'desvio': np.std(Pacerto),
                           'mediana': np.median(Pacerto),
                           'minimo': np.min(Pacerto),
                           'maximo': np.max(Pacerto),
                           'tempo': fim - inicio}




Separação 20/80 e distância de Minkowski de ordem 0.5



Separação 20/80 e distância de Minkowski de ordem 0.6666666666666666



Separação 20/80 e distância de Minkowski de ordem 1



Separação 20/80 e distância de Minkowski de ordem 1.5



Separação 20/80 e distância de Minkowski de ordem 2



Separação 20/80 e distância de Minkowski de ordem 2.5



Separação 30/70 e distância de Minkowski de ordem 0.5



Separação 30/70 e distância de Minkowski de ordem 0.6666666666666666



Separação 30/70 e distância de Minkowski de ordem 1



Separação 30/70 e distância de Minkowski de ordem 1.5



Separação 30/70 e distância de Minkowski de ordem 2



Separação 30/70 e distância de Minkowski de ordem 2.5



Separação 50/50 e distância de Minkowski de ordem 0.5



Separação 50/50 e distância de Minkowski de ordem 0.6666666666666666



Separação 50/50 e distância de Minkowski de ordem 1



Separação 50/50 e distância de Minkowski de ordem 1.5



Separação 50/50 e distância de Minkowski de ordem 2

In [5]:
for i in resultados.keys():
    print(f'\n\nResultados para a separação {i}\n')
    print('Ordem da distância | Média  | Desvio Padrão | Mediana | Mínimo | Máximo | Tempo (s)')
    print('-------------------|--------|---------------|---------|--------|--------|----------')
    for j in resultados[i].keys():
        r = resultados[i][j]
        print(f'       {j:.4f}      | {r["media"]:.4f} |    {r["desvio"]:.4f}     |  {r["mediana"]:.4f} | {r["minimo"]:.4f} | {r["maximo"]:.4f} |  {r["tempo"]:.2f}')



Resultados para a separação 20/80

Ordem da distância | Média  | Desvio Padrão | Mediana | Mínimo | Máximo | Tempo (s)
-------------------|--------|---------------|---------|--------|--------|----------
       0.5000      | 0.7523 |    0.0262     |  0.7530 | 0.6802 | 0.8057 |  51.00
       0.6667      | 0.7617 |    0.0299     |  0.7611 | 0.6923 | 0.8219 |  49.97
       1.0000      | 0.7749 |    0.0240     |  0.7733 | 0.7126 | 0.8340 |  46.24
       1.5000      | 0.7790 |    0.0297     |  0.7814 | 0.6883 | 0.8502 |  49.81
       2.0000      | 0.7811 |    0.0253     |  0.7854 | 0.7287 | 0.8462 |  43.40
       2.5000      | 0.7775 |    0.0279     |  0.7814 | 0.7085 | 0.8381 |  51.84


Resultados para a separação 30/70

Ordem da distância | Média  | Desvio Padrão | Mediana | Mínimo | Máximo | Tempo (s)
-------------------|--------|---------------|---------|--------|--------|----------
       0.5000      | 0.7611 |    0.0272     |  0.7616 | 0.6991 | 0.8241 |  67.34
       0.6667      | 0.

### Classificador Distância Mínima ao Centróide

In [229]:
C = conjunto.iloc[:, -1].unique()
for s in sep.keys():

    print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        conjunto = conjunto.sample(frac=1).reset_index(drop=True) # embaralha o conjunto
        x, y = conjunto.iloc[:, :-1], conjunto.iloc[:, -1]

        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn+1:, :], y.iloc[N_trn+1:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.mean()

        acerto = 0 # contador de acertos
        for i in x_tst.index:
            x_new = np.array(x_tst.loc[i, :]) # nova amostra a ser classificada
            y_new = y_tst.loc[i] # classe da nova amostra

            dist_array = []
            for k in C:
                dist_array.append(minkowski(x_new, np.array(centroides[k]), p=2)) # distância Euclidiana ao centróide da classe k
            
            k_min = np.argmin(dist_array) # índice do menor valor de distância

            if C[k_min] == y_new:
                acerto += 1 # verifica se a classificação foi correta
        
        Pacerto.append(acerto / len(x_tst))

    fim = time.perf_counter()

    resultados[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'tempo': fim - inicio}



Separação 20/80


Separação 30/70


Separação 50/50


Separação 70/30


Separação 80/20


In [ ]:
centroides = {}
for classe in C:
    df_classe = conjunto[conjunto.iloc[:, -1] == classe].iloc[:N_trn, :-1]centroides[classe] = df_classe.mean()

SyntaxError: invalid syntax (4118691873.py, line 3)

### Classificador Distância Mínima ao Centróide (versão robusta a outliers)

### Classificador de Máxima Correlação

In [227]:
C = conjunto.iloc[:, -1].unique()
for s in sep.keys():

    print(f'\n\nSeparação {s}')

    N_trn = round(sep[s]['train'] * len(conjunto)) # número de amostras do conjunto de treino

    Pacerto = [] # lista que armazena a porcentagem de acertos em cada rodada
    inicio = time.perf_counter()

    for rep in range(100): # 100 rodadas de treino
        #print(f'Rodada de número {rep + 1}')

        conjunto = conjunto.sample(frac=1).reset_index(drop=True) # embaralha o conjunto
        x, y = conjunto.iloc[:, :-1], conjunto.iloc[:, -1]

        # conjunto de treino
        x_trn, y_trn = x.iloc[:N_trn, :], y.iloc[:N_trn]  
        
        # conjunto de teste
        x_tst, y_tst = x.iloc[N_trn+1:, :], y.iloc[N_trn+1:]

        centroides = {}
        for classe in C:
            df_classe = x_trn[y_trn == classe]
            centroides[classe] = df_classe.mean()

        acerto = 0 # contador de acertos
        for i in x_tst.index:
            x_new = np.array(x_tst.loc[i, :]) # nova amostra a ser classificada
            y_new = y_tst.loc[i] # classe da nova amostra

            dist_array = []
            for k in C:
                dist_array.append(minkowski(x_new, np.array(centroides[k]), p=2)) # distância Euclidiana ao centróide da classe k
            
            k_min = np.argmin(dist_array) # índice do menor valor de distância

            if C[k_min] == y_new:
                acerto += 1 # verifica se a classificação foi correta
        
        Pacerto.append(acerto / len(x_tst))

    fim = time.perf_counter()

    resultados[s] = {'media': np.mean(Pacerto),
                    'desvio': np.std(Pacerto),
                    'mediana': np.median(Pacerto),
                    'minimo': np.min(Pacerto),
                    'maximo': np.max(Pacerto),
                    'tempo': fim - inicio}



Separação 20/80


Separação 30/70


KeyboardInterrupt: 

In [ ]:
conjunto.iloc[:N_trn, :][conjunto.iloc[:N_trn, -1] == classe]